# Схемы валидации

## Imports

In [2]:
import pandas as pd
import numpy as np

## Homework

Исходные данные - Yandex Cup 2022 RecSys:
- Описание соревнования - https://contest.yandex.ru/yacup/contest/41618/problems/
- Данные - https://disk.yandex.ru/d/SI1aAooPn9i8TA
- Описание данных - в архиве likes_data.zip три файла:
  - train - обучающий набор данных. Каждая строка - последовательность id треков, которые лайкнул один пользователь. Гарантируется, что лайки даны в той последовательности, в которой их ставил пользователь.
  - test - набор тестовых данных. Имеет точно такой же формат, но в каждой строке не хватает последнего лайка, который надо предсказать.
  - track_artists.csv - информация о исполнителях треков. Гарантируется, что у каждого трека есть ровно один исполнитель. Для треков, у которых фактически несколько исполнителей, мы оставили того, который считается основным исполнителем трека.
- Описание сабмита - в качестве решения необходимо отправить файл, в котором для каждого пользователя из test в отдельной строке будет не более 100 треков, разделенных пробелом. Гарантируется, что у каждого пользователя будет только 1 лайк в тесте
- Метрика - MRR@100

Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

**Итоговая задача** - построить схему валидации для данного соревнования с учетом особенностей сорвенования
- Между `train` и `test` не должно быть общих пользователей
- Количество фолдов задается через параметр класса `n_folds`
- В `test` должно быть не более `p` последних треков (параметр класса `p`)

In [4]:
yandex_data = pd.read_csv('/kaggle/input/recsys-yandex-cup-2022/dataframe.csv/dataframe.csv') # Подгружаем данные

In [5]:
yandex_data.head()

,user_id,track_id,rank,reversed_rank,is_train
0,0,333396,0,53,1
1,0,267089,1,52,1
2,0,155959,2,51,1
3,0,353335,3,50,1
4,0,414000,4,49,1


In [6]:
yandex_data = yandex_data.drop(['rank', 'reversed_rank', 'is_train'], axis=1)

In [7]:
yandex_data = yandex_data.rename({'user_id': 'user', 'track_id': 'item'}, axis=1)

In [8]:
# Предобработка и добавление порядка
yandex_data['order'] = yandex_data.groupby('user')['item'].cumcount(ascending=False) 

In [11]:
yandex_data.head()

,user,item,order
94188653,0,471705,0
94188652,0,219426,1
94188651,0,101168,2
94188650,0,361110,3
94188649,0,22932,4


In [12]:
class UsersKFoldPOut:
    def __init__(
        self,
        n_folds: int,
        p: int,
        rank_col: str = 'order',
        user_column: str = 'user',
        random_seed: int = 23
    ):
        self.n_folds = n_folds
        self.p = p
        self.rank_col = rank_col
        self.user_column = user_column
        self.random_seed = random_seed


    def split(self, df):
        users = df[self.user_column].unique()
        users_count = len(users)

        np.random.seed(self.random_seed)
        np.random.shuffle(users)

        fold_sizes = np.full(self.n_folds, users_count // self.n_folds, dtype=int)
        fold_sizes[: users_count % self.n_folds] += 1
        current = 0
        for fold_size in fold_sizes:
            start, stop = current, current + fold_size
            test_fold_users = users[start:stop]
            test_mask = df[self.user_column].isin(test_fold_users)
            train_mask = ~test_mask
            # Берем только те сэмплы, для которых порядок меньше p
            test_mask = test_mask & (df[self.rank_col] < self.p)

            yield train_mask, test_mask

In [13]:
par = 5
cv = UsersKFoldPOut(n_folds=3, p=par)

for i, (train_mask, test_mask) in enumerate(cv.split(yandex_data)):
    train = yandex_data[train_mask]
    test = yandex_data[test_mask]
    # Проверка, что нет пересечения train и test
    assert set() == set(train.user).intersection(test.user)
    # Проверка, что нет рангов больше p
    assert test[test.order > par].shape[0] == 0

    print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')


Fold#0 | Train: 78277912, Test: 1933475
Fold#1 | Train: 78277912, Test: 1933475
Fold#2 | Train: 78277978, Test: 1933470
